# Pipeline Reuse

This notebook shows how to reuse an existing pipeline.  It assumes that the pipeline created by the summarization_pipeline script in this directory has been run, just past where the pipeline has been uploaded.

From the Kubeflow dashboard, you should be able to find the pipeline in the 'pipelines' list. Create a run, using the summarize_exp experiment, and the following parameters:

* source_repo: https://github.com/ntl-ibm/kubeflow-ppc64le-examples.git
* source_branch: 3.0.0
* source_context: natural-language-processing/huggingface-summarization/src
* minio_endpoint: minio-service.kubeflow:9000
* checkpoint: google/pegasus-xsum
* model_max_length: 512
* model_version: 1
* epochs: 3
* model_name: pegasus-xsum
* prefix:
* suffix:


Training the model will take up to 20 minutes. When it completes, you can run inferences against the summarizer that is trained with the pegasus-xsum base model.

In [1]:
import requests
import json
import subprocess
import yaml

## Retrieve the inference URL

In [6]:
p = subprocess.run(
    "oc get inferenceservice pegasus-xsum -o yaml", shell=True, stdout=subprocess.PIPE
)

desc = yaml.safe_load(p.stdout)
url = desc["status"]["address"]["url"]
url

'http://pegasus-xsum.ntl-us-ibm-com.svc.cluster.local/v1/models/pegasus-xsum:predict'

## Sample text

In [24]:
text = "The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs.  It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."
  
import textwrap

for line in textwrap.wrap(text, width=70):
    print(line)

The Inflation Reduction Act lowers prescription drug costs, health
care costs, and energy costs.  It's the most aggressive action on
tackling the climate crisis in American history, which will lift up
American workers and create good-paying, union jobs across the
country. It'll lower the deficit and ask the ultra-wealthy and
corporations to pay their fair share. And no one making under $400,000
per year will pay a penny more in taxes.


## Inference

In [25]:
r = requests.post(url,
              json=
                  {"instances" : [text]}
              )

In [26]:
for line in textwrap.wrap(r.json()["summary"], width=70):
    print(line)

President Obama has signed into law one of the biggest tax cuts in
history.


## Hallucination

The summary returned is clearly not correct. The original text says nothing about tax cuts, nor does it mention President Obama or signing the act into law.

This is called a Hallucination, as the result appears legit, but is actually totally incorrect.

Real world solutions would likely need to include human feedback into the training and evaluation to lower the risk of this type of problem happening in practice.